# Context

LZ4 is a widely-used lossless compression algorithm. The reference
implementation of LZ4 at github.com/lz4/lz is subject to a heap-based
buffer overflow in releases prior to 1.9.2 as described in
CVE-2019-17543. The vulnerability is fixed by commit
d7cad81093cd805110291f84d64d385557d0ffba.


# Aim

Lets try to determine all the code that may still be vulnerable!

First lets define WoC access functions

In [54]:
import requests, json

def showCnt (type, sha1):
  url='http://worldofcode.org/api/lookup?command=showCnt' + '&type='+ type + '&sha1=' + sha1
  r = requests.get(url)
  res = json.loads(r.content)['stdout']
  if type == 'commit': 
    res = res.split(';')
    return ('Tree:'+res[1]+'\nParent:'+res[2]+'\nAuthor:'+res[3]+'\n')
  return (res)


def getValue (map, key):
  #map/key may not be correct names of the parameters
  url='http://worldofcode.org/api/lookup?command=getValues' + '&type='+ map + '&sha1=' + key
  r = requests.get(url)
  return (json.loads(r.content)['stdout']).split(';')[1:]

Once these functions are defined we can investigate the the problem:

What is the commit?

What files does it modify?

In [55]:
print (showCnt('commit', 'd7cad81093cd805110291f84d64d385557d0ffba'))

v = getValue('c2b', 'd7cad81093cd805110291f84d64d385557d0ffba')
print ('c2b no of elements:' + str(len(v)))
for b in v: print (b+'\n')

v = getValue('c2f', 'd7cad81093cd805110291f84d64d385557d0ffba')
print ('c2f no of elements:' + str(len(v)))
for f in v: print (f+'\n')             

Tree:9af7fb7a0b32809791cad70c12eda3dc9ccb48c7
Parent:1bcde6414a68094601ecd57a968808fdd43fb986
Author:Nick Terrell <terrelln@fb.com>

c2b no of elements:1
9808d70aed03290c648b983ea404446779eff501


c2f no of elements:1
lib/lz4.c




The commit with the fix contains only one blob
(9808d70aed03290c648b983ea404446779eff501) that creates a new version of lib/lz4.c. The author of that commit is
Nick Terrell <terrelln@fb.com>. 

What projects fixed the vulerability? If they have fixed blob, then they should be OK, 
if they have the same commit, they are also probably forks.
We use p to represent original repo and P to represent deforked repo (one among repos that 
share lots of commits (https://arxiv.org/abs/2002.02707)

In [56]:
v = getValue('b2P', '9808d70aed03290c648b983ea404446779eff501')
print ('b2P no of elements:' + str(len(v)))
for p in v: print (p+'\n')

v = getValue('c2P', 'd7cad81093cd805110291f84d64d385557d0ffba')
print ('c2P no of elements:' + str(len(v)))
for p in v: print (p+'\n')

v = getValue('c2p', 'd7cad81093cd805110291f84d64d385557d0ffba')
print ('c2p no of elements:' + str(len(v)))
for p in v: print (p+'\n')


b2P no of elements:26
0-wiz-0_libuv

06094051_librdkafka

1010101012101_borg

3370sohail_gecko-dev

540KJ_root

6180_python-lz4

Alexhuszagh_c-blosc

Bambooie_gdsfmt

cactus74_fst

darkskygit_simple_kv

draede_cx

dudw_libportable

fangq_matzip

girdharshubham_gokafka

git.bioconductor.org_packages_gdsfmt

jmoiron_golz4

karubabu_quickbms

liliilli_Dy

lmtwga_lzbench

marcelorl_graphql-real-time-example

pharaoh1_7-Zip-zstd

scalarwaves_zbox

silnrsi_grcompiler

tafia_fstlib

ushiiwaka_ps2-packer

windreamer_py-lz4framed


c2P no of elements:1
0-wiz-0_libuv


c2p no of elements:7
MiniclipPortugal_lz4

bobby0809_lz4

gitlab.com_exokos_data/LibLZ4

gitlab.com_freedesktop-sdk_mirrors/github/lz4/lz4

gitlab.com_fuchsia-cn_fuchsia/third_party/lz4

lz4_lz4

terrelln_lz4




## Results 

b2P shows 26 (deforeked repos) projects has the fixed code. They are 13862 distinct repos (cveb2p.txt)

c2P shows one project to have the fixing commit (that single project has 7681 forks(repos)). 

c2p shows that only seven actual repos to have the exact commit.  

Somehow the fix was produced via different commits in the forks and 
many other projects (26) have implemented the fix even though these are not forks.
 
                       
Now lets try to identify the code that may still be vulnerable. How?
By looking at the pre-fix file content.                       

In [57]:
v = getValue('b2ob', '9808d70aed03290c648b983ea404446779eff501')
print ('b2ob no of elements:' + str(len(v)))
for b in v: print (b+'\n')

b2ob no of elements:9
08cf6b5cd72b8182552dcc53bdc0d83ccd5382fd

143c36e1a7448c488a44498ac953ea222f3f38d0

4046102e6deea607dc12f870c14295cab1efee77

707b94c41954792f95b6bb2d316b787352969cef

877d14edad4b0568598d64579ecb68db82bd59f4

c9c5a072a193b9b7f7c010797d0e122038587332

e51a3e0a46c9608bedbb0b9565d736240b30bde6

e614c4577f2ae8b2db76ff838f2051eeeeb1a89b

ed928ced3f154ab414f657c4dbd0193cbe7cd969




# Results

In fact, if we proceed to find parent blobs (see code below), we find recursively 514 unique old blobs (cveOBlobs.txt) in 706 different projects (cveOBlobsPs.txt).

We then need to identify those that do not have the fixed blob in order to determine projects that 
might contain still-vulnerable code.

One way to do that is to look at the set difference of projects containing fixed blob and ones that lack it, but have one of these 514 paretn blobs.

There are 433185 repos for the 706 different projects (cveP2p.txt) containin old blobs.
To be more precise, we can trace the old blobs to commits (cveOBlobsCs.txt) and commits to a much smaller number of  repos (cveOBlobsCsps.txt). 

Similarly, the fixed blob maps to 64 commits (cveBlobCs.txt) and these to 104 repos (cveBlobCsps.txt). This means that in at least 40152 - 104 repos the vulnerability was either not fixed or fixed in a different way!


# Appendix

The following code finds all predecessor blobs and projects that contain them. For convenience (running the script takes some time).

In [ ]:
#
# Recursively walk old blobs using b2ob and print all old blobs and all
# projects associated with each blob. When finished, blob_list will
# contain the list of all unique blobs and proj_list will contain all
# unique projects (from b2P).
#
def b2ob_r (blob, blob_list, proj_list):
    blob = blob.rstrip()
    if blob in blob_list:
        # skip duplicates
        return
    blob_list.append(blob)
    print (blob)
    v = getValue ('b2P', blob)
    for p in v:
        p = p.rstrip()
        print('  proj = ' + p)
        if p not in proj_list:
            proj_list.append(p)
    v = getValue ('b2ob', blob)
    for b in v:
        b2ob_r (b, blob_list, proj_list)


blob_list = []
proj_list = []
b2ob_r('9808d70aed03290c648b983ea404446779eff501', blob_list, proj_list)

print('')
print('number of blobs: ' + str(len(blob_list)))
print('number of projects: ' + str(len(proj_list))) 